# TOKENIZE 

In [97]:
# My code for int/float vars:
import string
import re

#  FUNCTIONS FOR TOKENIZING STATEMENT INPUTTED TO EXECUTE:

def float_int_check(query, tokens):
    nums = []
    x = re.findall(r'-?\d+\.\d+', query)
    # If the number is a float,
    if x:
        if query[0] in x[0]:
            tokens.append(float(x[0]))
            query = query[len(x[0]):]
        else:
            # If the list wasn't empty but number is integer:
            nums = collect_characters(query, string.digits)
            tokens.append(int(nums))
            query = query[len(nums):]
            
    # If the number is an integer:
    if len(x) == 0:
        nums = collect_characters(query, string.digits)
        tokens.append(int(nums))
        query = query[len(nums):]
    return query


def collect_characters(query, allowed_characters):
    letters = []
    for letter in query:
        if letter not in allowed_characters:
            break
        letters.append(letter)
    return "".join(letters)


def remove_leading_whitespace(query, tokens):
    whitespace = collect_characters(query, string.whitespace)
    return query[len(whitespace):]


def remove_word(query, tokens):
    word = collect_characters(query,
                              string.ascii_letters + "_" + string.digits)
    if word == "NULL":
        tokens.append(None)
    else:
        tokens.append(word)
    return query[len(word):]


def remove_text(query, tokens):
    assert query[0] == "'"
    query = query[1:]
    end_quote_index = query.find("'")
    text = query[:end_quote_index]
    tokens.append(text)
    query = query[end_quote_index + 1:]
    return query


def tokenize(query):
    tokens = []
    while query:
        old_query = query

        if query[0] in string.whitespace:
            query = remove_leading_whitespace(query, tokens)
            continue

        if query[0] in (string.ascii_letters + "_"):
            query = remove_word(query, tokens)
            continue

        if query[0] in "(),;":
            tokens.append(query[0])
            query = query[1:]
            continue
            
        if query[0] == "*":
            tokens.append(query[0])
            query = query[1:]
            continue

        if query[0] == "'":
            query = remove_text(query, tokens)
            continue

        # INTEGERS AND FLOATS
        if query[0] in string.digits or query[0] == "-":
            query = float_int_check(query, tokens)
            continue

        if len(query) == len(old_query):
            raise AssertionError("Query didn't get shorter.")
        else:
            print("Not in the if statements")

    return tokens

# DATABASE CLASS

In [111]:
all_database = {}
class Database(object):
    def __init__(self):
        self.database = {}

    def create_table(self, tokens):
        # Temp storage:
        name_cols = []
        actual_table = {}
        d_type = []
        
        # I want to pull all the lowercase text from the tokens,
        # These will be the table and column names of the table. (for CREATE TABLE)
        for i in range(len(tokens)):
            if tokens[i].islower():
                # We also want the data type of the columns:
                d_type.append(tokens[i+1])
                # Append column names:
                name_cols.append(tokens[i])
        # d_type will have a bracket because of the title, so we pop it off.
        d_type.pop(0)
                              
                
        # Extract table name:
        title = name_cols[0]
        # Add columns to table:
        for i in name_cols[1:]:
            actual_table[i] = []
            
        # Insert table into database:
        #self.database[title] = [actual_table]
        
        # Using global variable instead to prevent overwriting.
        all_database[title] = [actual_table]

        return 
    
    def insert_vals(self,tokens):
        # Temp storage:
        my_list = []
        # In an INSERT INTO statement, the third token is the table name.
        table_name = tokens[2]
        table = db.get_table(table_name)
        keys = list(table.keys())
            
        # Extract values, I will turn them into a tuple in my return. 
        # The INSERT INTO statement values begin after the table name and bracket, so we start at [5:]
        for i in tokens[5:]:
            if i != ")":
                if i != "," and i != ";":
                    my_list.append((i))
                        
        # Inserting the values
        # The columns I believe are the same amoutn as the values inserted, so lets try this:
        for i in range(len(my_list)):
            table[keys[i]].append(my_list[i])
        
                    
        return     
        #return self.database[table_name]
        
                
    def get_table(self, table_id):
        # Since I used nested dictionaries, my table will first pull as a list.
        #table_list = self.database[table_id]
        
        # We are going to use a global variable instead to prevent overwriting.
        table_list = all_database[table_id]
        
        # Index at 0 to pull entire table.
        return table_list[0]
    
    def get_rows(self,tokens,table):
        # Temp storage
        cols = []
        col_to_sort = []
        all_rows = []
        
        # What are we selecting?
        
        for i in range(len(tokens)):
            # After SELECT are the column names we need.
            if tokens[i] == "SELECT":
                select_cols = tokens[i+1:]
                
        # We need to slice, everything after the column names we dont want in the list.
        remove_from = select_cols.index("FROM")
        select_cols = select_cols[:remove_from]
        # Remove commas/special characters
        for i in select_cols:
            if i != ",":
                cols.append(i)
        
        # What are we ordering it by?
        for i in range(len(tokens)):
            if tokens[i] == "BY":
                order = tokens[i+1:]
        for i in order:
            if i != ";" and i != "," and i != ")":
                col_to_sort.append(i)
                
        # Get all rows:
        for key, value in table.items():
            all_rows.append(table[key])
            
        # Transpose list 
        t_list = [tuple(i) for i in zip(*all_rows)]
        
        
        # Order by, we need to fix this somehow!
        t_list.sort(key = lambda i: i[0])
        
        # Return tuple
        return t_list
    


CREATE: dict_items([('students', [{'col1': [], 'col2': [], 'col3': []}])])


# TESTING THE DATABASE CLASS AFTER USING GLOBAL VAR:

In [120]:
# TEST CREATE:
tokens =  ['CREATE', 'TABLE', 'students', '(', 'col1', 'INTEGER', ',', 'col2', 'TEXT', ',', 'col3', 'REAL', ')', ';']
db = Database()
db.create_table(tokens)
print("CREATE:",all_database.items())


CREATE: dict_items([('students', [{'col1': [], 'col2': [], 'col3': []}])])


In [121]:
# TEST INSERT:
query = "INSERT INTO students VALUES (7842, 'string with spaces', 3.0);"
tokens = tokenize(query)
db.insert_vals(tokens)
print("INSERT:",all_database.items())

INSERT: dict_items([('students', [{'col1': [7842], 'col2': ['string with spaces'], 'col3': [3.0]}])])


In [122]:
# INSERT TWICE:
query_f = "INSERT INTO students VALUES (7, 'look a null', NULL);"
tokens = tokenize(query_f)
db.insert_vals(tokens)
print("INSERT TWO:",all_database.items())

INSERT TWO: dict_items([('students', [{'col1': [7842, 7], 'col2': ['string with spaces', 'look a null'], 'col3': [3.0, None]}])])


In [123]:

# INSERT THRICE:
query3 = "INSERT INTO students VALUES (3, 'hi', 4.5);"
tokens = tokenize(query3)
db.insert_vals(tokens)
print("INSERT THREE:",all_database.items())


INSERT THREE: dict_items([('students', [{'col1': [7842, 7, 3], 'col2': ['string with spaces', 'look a null', 'hi'], 'col3': [3.0, None, 4.5]}])])


In [124]:

# Testing get_table
table = db.get_table("students")
print("GET TABLE:",table)


GET TABLE: {'col1': [7842, 7, 3], 'col2': ['string with spaces', 'look a null', 'hi'], 'col3': [3.0, None, 4.5]}


In [109]:


# TEST SELECT:
select_stat = "SELECT col1, col2, col3 FROM students ORDER BY col1;"
tokens = tokenize(select_stat)
print("GET ROWS",db.get_rows(tokens,table))

GET ROWS [(7842, 'string with spaces', 3.0), (7842, 'string with spaces', 3.0)]


# EXECUTE TEST

In [132]:
# Create:
query1 = "CREATE TABLE students (col1 INTEGER, col2 TEXT, col3 REAL);"
# Inserts:
query = "INSERT INTO students VALUES (7842, 'string with spaces', 3.0);"
query_f = "INSERT INTO students VALUES (7, 'look a null', NULL);"
query3 = "INSERT INTO students VALUES (3, 'hi', 4.5);"
# Select:
select_stat = "SELECT col1, col2, col3 FROM students ORDER BY col1;"



def execute(statement):
    """
    Takes a SQL statement.
    Returns a list of tuples (empty unless select statement
    with rows to return).
    """
    # Create instance of DataBase Class:
    db = Database()
    
    # Only SELECT statements should return rows/table.
    # Tokens that should return an empty list:
    c = "CREATE"
    ins = "INSERT"
    
    # First we tokenize,
    tokens = tokenize(statement)
    # If it's a create statement,
    if c in tokens:
        db.create_table(tokens)
        return []
    # If it's an insert statement:
    elif ins in tokens:
        db.insert_vals(tokens)
        return []
    # If it is a select statement:
    else:
        # Table ID is right after FROM statement.
        # We have to pull here so we can have the table inputted into the get_rows func.
        for i in range(len(tokens)):
            if tokens[i] == "FROM":
                table_name = tokens[i+1]
        table = db.get_table(table_name)
        # USE GET ROWS FO THE REST! 
        return db.get_rows(tokens,table)
        
# Create
execute(statement = query1)
print("CREATE:",all_database)
# Insert
execute(statement = query)
execute(statement = query_f)
execute(statement = query3)
print("INSERT:",all_database)

# Select
print(execute(statement = select_stat))

CREATE: {'students': [{'col1': [], 'col2': [], 'col3': []}]}
INSERT: {'students': [{'col1': [7842, 7, 3], 'col2': ['string with spaces', 'look a null', 'hi'], 'col3': [3.0, None, 4.5]}]}
[(3, 'hi', 4.5), (7, 'look a null', None), (7842, 'string with spaces', 3.0)]


# TEST DATABASE CLASS WITHOUT EXECUTE:


In [136]:

# Create table"
tokens =  ['CREATE', 'TABLE', 'students', '(', 'col1', 'INTEGER', ',', 'col2', 'TEXT', ',', 'col3', 'REAL', ')', ';']
db = Database()
db.create_table(tokens)

# Insert vals:
query = "INSERT INTO students VALUES (7842, 'string with spaces', 3.0);"
tokens = tokenize(query)
db.insert_vals(tokens)
# Insert more values:
query_f = "INSERT INTO students VALUES (7, 'look a null', NULL);"
tokens = tokenize(query_f)
db.insert_vals(tokens)
query3 = "INSERT INTO students VALUES (3, 'hi', 4.5);"
tokens = tokenize(query3)
db.insert_vals(tokens)

print("DATABASE AS IS:", all_database)



DATABASE AS IS: {'students': [{'col1': [7842, 7, 3], 'col2': ['string with spaces', 'look a null', 'hi'], 'col3': [3.0, None, 4.5]}]}


# DIFFERENT TABLE TEST:

In [146]:
create_q = "CREATE TABLE student (name TEXT, grade REAL, piazza INTEGER);"
insert_q1 = "INSERT INTO student VALUES ('James', 4.0, 1);"
insert_q2 = "INSERT INTO student VALUES ('Yaxin', 4.0, 2);"
insert_q3 = "INSERT INTO student VALUES ('Li', 3.2, 2);"
select_stat = "SELECT * FROM student ORDER BY piazza, grade;"

In [149]:
all_database = {}

# Create table"
tokens = tokenize(create_q)
db = Database()
db.create_table(tokens)

# Insert vals:
tokens = tokenize(insert_q1)
db.insert_vals(tokens)

# Insert more values:
tokens = tokenize(insert_q2)
db.insert_vals(tokens)

# insert more vals:
tokens = tokenize(insert_q3)
db.insert_vals(tokens)

print("DATABASE AS IS:", all_database)

# Testing get_rows, AKA SELECT!
tokens = tokenize(select_stat)
table = db.get_table("student")
print(db.get_rows(tokens,table))

DATABASE AS IS: {'student': [{'name': ['James', 'Yaxin', 'Li'], 'grade': [4.0, 4.0, 3.2], 'piazza': [1, 2, 2]}]}
[('James', 4.0, 1), ('Li', 3.2, 2), ('Yaxin', 4.0, 2)]


# FIXING SELECT ORDER BY  STATEMENT
- Since Execute contains the get table name portion, we have to run the get_rows individually:

In [152]:
def get_rows(self,tokens,table):
        # Temp storage
        cols = []
        col_to_sort = []
        all_rows = []
        
        # What are we selecting?
        
        for i in range(len(tokens)):
            # After SELECT are the column names we need.
            if tokens[i] == "SELECT":
                select_cols = tokens[i+1:]
                
        # We need to slice, everything after the column names we dont want in the list.
        remove_from = select_cols.index("FROM")
        select_cols = select_cols[:remove_from]
        # Remove commas/special characters
        for i in select_cols:
            if i != ",":
                cols.append(i)
        
        # What are we ordering it by?
        for i in range(len(tokens)):
            if tokens[i] == "BY":
                order = tokens[i+1:]
        for i in order:
            if i != ";" and i != "," and i != ")":
                col_to_sort.append(i)
                
        # Get all rows:
        for key, value in table.items():
            all_rows.append(table[key])
            
        # Transpose list 
        t_list = [tuple(i) for i in zip(*all_rows)]
        
        
        # Order by, we need to fix this somehow!
        t_list.sort(key = lambda i: i[0])
        
        # Return tuple
        return t_list
    

# Testing get_rows, AKA SELECT!

# Create table"
tokens = tokenize(create_q)
db = Database()
db.create_table(tokens)

# Insert vals:
tokens = tokenize(insert_q1)
db.insert_vals(tokens)

# Insert more values:
tokens = tokenize(insert_q2)
db.insert_vals(tokens)

# insert more vals:
tokens = tokenize(insert_q3)
db.insert_vals(tokens)

print("DATABASE AS IS:", all_database)

# Testing get_rows, AKA SELECT!
tokens = tokenize(select_stat)
table = db.get_table("student")
print("Table returned:",table)
print("ROWS RETURNED:",db.get_rows(tokens,table))

expected = [('James', 4.0, 1),('Li', 3.2, 2),('Yaxin', 4.0, 2)]
print("EXPECTED RETURN:", expected)


DATABASE AS IS: {'student': [{'name': ['James', 'Yaxin', 'Li'], 'grade': [4.0, 4.0, 3.2], 'piazza': [1, 2, 2]}]}
Table returned: {'name': ['James', 'Yaxin', 'Li'], 'grade': [4.0, 4.0, 3.2], 'piazza': [1, 2, 2]}
ROWS RETURNED: [('James', 4.0, 1), ('Li', 3.2, 2), ('Yaxin', 4.0, 2)]
EXPECTED RETURN: [('James', 4.0, 1), ('Li', 3.2, 2), ('Yaxin', 4.0, 2)]


In [155]:
# Testing using execute:
create_q = "CREATE TABLE student (name TEXT, grade REAL, piazza INTEGER);"
insert_q1 = "INSERT INTO student VALUES ('James', 4.0, 1);"
insert_q2 = "INSERT INTO student VALUES ('Yaxin', 4.0, 2);"
insert_q3 = "INSERT INTO student VALUES ('Li', 3.2, 2);"
select_stat = "SELECT * FROM student ORDER BY piazza, grade;"

# Create
execute(statement = create_q)
print("CREATE:",all_database)

# Insert
execute(statement = insert_q1)
execute(statement = insert_q2)
execute(statement = insert_q3)
print("INSERT:",all_database)

# Select
print("MY RETURN:",execute(statement = select_stat))

expected = [('James', 4.0, 1),('Li', 3.2, 2),('Yaxin', 4.0, 2)]
print("EXPECTED RETURN:", expected)

CREATE: {'student': [{'name': [], 'grade': [], 'piazza': []}]}
INSERT: {'student': [{'name': ['James', 'Yaxin', 'Li'], 'grade': [4.0, 4.0, 3.2], 'piazza': [1, 2, 2]}]}
MY RETURN: [('James', 4.0, 1), ('Li', 3.2, 2), ('Yaxin', 4.0, 2)]
EXPECTED RETURN: [('James', 4.0, 1), ('Li', 3.2, 2), ('Yaxin', 4.0, 2)]


# ALL ORIGINAL TESTS BELOW: CREATE TABLE, INSERT VALS, GET TABLE, GET ROWS

In [80]:
# Testing table creation:
tokens =  ['CREATE', 'TABLE', 'students', '(', 'col1', 'INTEGER', ',', 'col2', 'TEXT', ',', 'col3', 'REAL', ')', ';']
db = Database()
test = db.create_table(tokens)
print("This is the database:",test)
table_list = test["students"]
print("When you use the table name to extract, it is a list.",table_list)
table = table_list[0]
print("If you index the list at [0], you can pull the table",table)
table['col1'] = 1
print("Then you can update the table using table[column name] = whatever.",table["col1"] )


This is the database: {'students': [{'col1': [], 'col2': [], 'col3': []}]}
When you use the table name to extract, it is a list. [{'col1': [], 'col2': [], 'col3': []}]
If you index the list at [0], you can pull the table {'col1': [], 'col2': [], 'col3': []}
Then you can update the table using table[column name] = whatever. 1


# TEST INSERT:

In [55]:
# TESTING CREATE/INSERT VALS:

# CREATE A TABLE INSTANCE:
tokens =  ['CREATE', 'TABLE', 'students', '(', 'col1', 'INTEGER', ',', 'col2', 'TEXT', ',', 'col3', 'REAL', ')', ';']
db = Database()
db.create_table(tokens)

# Insert vals test:
query = "INSERT INTO students VALUES (7842, 'string with spaces', 3.0);"
tokens = tokenize(query)
test_table = db.insert_vals(tokens)
print("Table:",test_table)

Table: {'col1': [7842], 'col2': ['string with spaces'], 'col3': [3.0]}


# TEST GET TABLE

In [57]:
# Testing get_table
tokens =  ['CREATE', 'TABLE', 'students', '(', 'col1', 'INTEGER', ',', 'col2', 'TEXT', ',', 'col3', 'REAL', ')', ';']
db = Database()
db.create_table(tokens)
db.get_table("students")

{'col1': [], 'col2': [], 'col3': []}

# MISC:

In [10]:
# Some queries to test:
# Query from test cases in project:
query1 = "CREATE TABLE students (col1 INTEGER, col2 TEXT, col3 REAL);"
query2 = "SELECT col1, col2, col3 FROM students ORDER BY col1;"
query = "INSERT INTO students VALUES (7842, 'string with spaces', 3.0);"
query_f = "INSERT INTO students VALUES (7, 'look a null', NULL);"

# Old Funcs: